In [ ]:
# function to write object and tags
from minio import Minio
from minio.commonconfig import Tags
from io import BytesIO
import os


def write_to_s3(bucket, filepath, tags={}, meta={}):
    # create Minio client and init bucket if necessary
    client = Minio(
        os.getenv("JUPYTERLAB_S3_ENDPOINT").replace("http://", ""),
        secure=False,
        access_key=os.getenv("JUPYTERLAB_S3_ACCESS_KEY_ID"),
        secret_key=os.getenv("JUPYTERLAB_S3_SECRET_ACCESS_KEY"),
    )
    found = client.bucket_exists(bucket)
    if not found:
        print("Bucket does not exist: %s" % (bucket))
        return False  # failure

    # init tags
    t = Tags(for_object=True)
    for k, v in tags.items():
        t[k] = v

    with open(filepath, mode="rb") as source_file:
        contents = source_file.read()

    client.put_object(
        bucket,
        filepath,
        data=BytesIO(contents),
        length=len(contents),
        tags=t,
        metadata=meta,
        content_type='application/csv'
    )
    return True  # success


write_to_s3(
    bucket="XXXX", #Update to match local environment
    filepath="pipeline-tx.parquet",
    tags={"color": "red", "size": "small"},
    meta={"cat": "meow", "dog": "woof", "horse": "neigh"})